# SmartFigure: User Guide

This notebook is the primary, example-driven user guide for **SmartFigure** — a small plotting helper for **SymPy expressions** in Jupyter, with a Plotly-first interactive widget.

**Audience:** mathematically-inclined researchers, students, and collaborators.

## Table of contents
- [What this package is for](#what-this-package-is-for-and-what-it-is-not)
- [Installation & Requirements](#installation--requirements)
- [Quickstart (5 minutes)](#quickstart-5-minutes)
- [Core Concepts](#core-concepts)
- [Common Tasks / Recipes](#common-tasks--recipes)
- [Applications](#applications-24-mini-projects)
- [Advanced Guide](#advanced-guide)
- [API Reference](#api-reference-lightweight)
- [Next Steps](#next-steps)


## What this package is for 

### Purpose
`SmartFigure` is designed for *small-to-medium* interactive exploration of **1D real-valued functions** defined symbolically in SymPy.

Typical use cases:
- Quickly sketch/inspect expressions while teaching or doing calculations.
- Explore *parameterized* families of functions with sliders.
- Keep workflows reproducible: the “state” is the SymPy expression + numeric parameter values.

### Mental model (Model / Controller / View)
- **Model:** `Plot` — holds a SymPy expression, sampling settings, and style.
- **Controller:** `SmartFigure` — owns plots, a viewport (`x_range`, `y_range`), and recomputation logic.
- **View:** `PlotBackend` — rendering seam (Plotly `FigureWidget` by default; Matplotlib optional).


## Installation & Requirements

### Minimal requirements
- Python 3.9+
- `numpy`
- `sympy`

### Optional dependencies
`gu_SmartFigure` intentionally *lazy-imports* heavy UI dependencies. Features appear when optional packages are present.

| Optional package | Install hint (generic) | Enables |
|---|---|---|
| `ipywidgets` | `pip install ipywidgets` | Jupyter widget container + sliders |
| `plotly` (with `FigureWidget`) | `pip install plotly` | Interactive Plotly-backed widget |
| `matplotlib` | `pip install matplotlib` | Script-friendly Matplotlib backend (`MplBackend`) |

### Environment notes
- **Google Colab is not supported** for the interactive widget.
- **JupyterLite/Pyodide** is supported *if* your build bundles the widget manager + Plotly.


`SmartFigure` is imported and made available by `gu_toolkit`

In [1]:
import gu_toolkit
gu_toolkit.setup() #SmartFigure imported

🔧 Initializing GU Toolkit...
✓ Matplotlib backend set to 'widget'
[gu_toolkit] Skip (disabled) gu_toolkit.plugins.example
[gu_toolkit] Loaded gu_toolkit.plugins.NamedFunction (exports=1, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartException.SmartException (exports=1, hook=True)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartFigure.SmartFigure (exports=9, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartParameters.SmartParameters (exports=4, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.numpify (exports=1, hook=False)
✅ Smart Exception Handler Activated.
✅ Smart Exception Handler Activated.
[gu_toolkit] Exported 951 names into the notebook namespace.
🎓 GU Toolkit Ready.


## Quickstart (5 minutes)

We’ll do a minimal, reproducible workflow:
1) create a `SmartFigure` without displaying immediately,
2) add a plot without a name,
3) add a plot without a parameter,
4) add a plot with a parameter
5) access existing plots by name
6) change the parameter value programmatically,
7) change the style of plotted functions
8) create a plot with a restricted domain or different sampling rate

#### 1) Create a `SmartFigure` without displaying immediately,


In [2]:
# By default: 
# var=x  the free variable is x
# x_range=(-4,4)  
# y_range=(-3,3)  
fig = SmartFigure() 

#### 2) Add a plot without a name

The figure automatically generates a name for you

In [8]:
fig.plot(1-x**2)  

<Plot 'f_1': 1 - x**2 | visible=True>

#### 3) Add a named plot without a parameter

In [13]:
fig.plot(sin(x**2), name="chirp")  

<Plot 'chirp': sin(x**2) | visible=True>

Named plots are idempotent. You can execute the plot command again with the same name. It will replace the plot.

In [14]:
fig.plot(sin(x**2), name="chirp")  

<Plot 'chirp': sin(x**2) | visible=True>

In [16]:
fig.plot(sin((x-0.5)**2), name="chirp")  

<Plot 'chirp': sin((x - 0.5)**2) | visible=True>

#### 4) Add a plot with a parameter


By default, a few single letter variable are defined as `Symbol`. Define a new symbol to use it as a parameter.

Since the expression depends on multiple variables,
we must specify the free symbol by passing plot the argument
`symbol=x`.

In [12]:
a=Symbol('a')
fig.plot(exp(-a * x**2), symbol=x, name="gaussian") 

<Plot 'gaussian': exp(-a*x**2) params=[a] | visible=True>

Expressions with the same symbol share the parameter value

In [25]:
fig.plot(
    exp(-a)-2*a*exp(-a)*(x-1),
    symbol=x, name="gaussian-tangent") 

<Plot 'gaussian-tangent': -2*a*(x - 1)*exp(-a) + exp(-a) params=[a] | visible=True>

#### 5) Access existing plots

You can access plots you have created by name

In [5]:
fig.plots['chirp']

<Plot 'chirp': sin(x**2) | visible=True>

Trying to access an undefined plot name gives an exception

In [ ]:
fig.plots['unknown_plot']

#### 6) Change the parameter value programmatically

Now you can use the slider to change the parameters.

To change the value of a parameter (say `a`) programmatically 
DO NOT DO
```python
a=1 #WRONG
```
as `a` is a formal symbol. Thange the value using the parameter registry:
```python
fig.parameter_registry[a].value=0.2 #CORRECT
```

In [26]:
fig.parameter_registry[a].value=0.2 

You can also get information about current values of parameters

In [13]:
display(fig.parameter_registry[a])

SmartParameter(id=a, value=0.19999999999999996, min_val=-1.0, max_val=1.0, step=0.01, default_val=0.0)

In [15]:
fig.parameter_registry[a].value

0.19999999999999996

#### 7) Changing styles
You can pass style when creating a plot, or mutate `plot.style` later.

`Style` is a small validated container:
- `visible: bool`
- `color: str | None`
- `width: float | None` (must be > 0)
- `opacity: float | None` (0..1)
- `linestyle: {'-', '--', ':', '-.'} | None`

**Gotchas**
- Invalid style values raise a `ValueError`/`TypeError` early.
- In a live widget, `plot.style.<field> = ...` triggers a redraw.


To change the style of existing plot set, obtain the plot and and set its style parameters

In [17]:
fig.plots['chirp'].style.linestyle='--'

#### 8) Create a plot with a restricted domain

Each `Plot` can sample on either:
- the figure viewport (use `domain=VIEWPORT` - default), or
- a plot-specific domain `(xmin, xmax)`.

Similarly, each plot can inherit the figure's sample count with `samples=VIEWPORT`.

**Gotchas**
- If a plot’s domain does not intersect the current viewport, it samples nothing (empty arrays).
- `domain` values are interpreted as floats; `None` can be used for “unbounded”, but unbounded domains will be clipped by the viewport.

In [29]:
fig.plot(sin(100*x), name="high_oscill", domain=(1, 2))

<Plot 'high_oscill': sin(100*x) | visible=True>

In [30]:
fig.plot(sin(100*x), name="high_oscill-low-res", domain=(-2, -1), samples=10)

<Plot 'high_oscill-low-res': sin(100*x) | visible=True>

## Common Tasks / Recipes

Short, searchable recipe cards.


In [ ]:
x = sp.Symbol("x")
fig = SmartFigure(var=x, show_now=False)
p = fig.plot(sp.sin(x), name="sine", style={"color": "green", "width": 2.0})

# Verify
assert p.name == "sine"
assert p.param_symbols == tuple()
p


### Recipe: Update an existing plot (same name)

Calling `fig.plot(..., name=<existing>)` updates the existing plot.


In [33]:
fig = SmartFigure()

fig.plot(sin(x), name="f")

<Plot 'f': sin(x) | visible=True>

In [35]:
fig.plot(cos(x), name="f")  # update in place

<Plot 'f': cos(x) | visible=True>

### Recipe: Hide/show a plot


In [37]:
fig = SmartFigure(var=x)
p = fig.plot(sin(x), name="f")


In [40]:
p.visible = False

In [41]:
p.visible = True

### Recipe: Remove a plot / clear the figure


In [63]:
fig = SmartFigure()

In [64]:
fig.plot(sin(x), name=r"sin(x)")
fig.plot(cos(x), name="b")

<Plot 'b': cos(x) | visible=True>

In [44]:
fig.remove("a")

In [51]:
fig.clear()

In [49]:
# This cell does not require the widget; it just illustrates the naming rule.
# If PlotlyBackend is available, these names appear in the legend.
good = r"$\sin(x)$ and cost \$5"
bad = r"$\sin(x)$ and then $oops"  # unmatched

print("Good label:", good)
print("Bad label :", bad)


Good label: $\sin(x)$ and cost \$5
Bad label : $\sin(x)$ and then $oops


In [ ]:
import csv
from pathlib import Path

x = sp.Symbol("x")
fig = SmartFigure(var=x, x_range=(-2, 2), samples=100, show_now=False)
p = fig.plot(sp.sin(x), name="sine")

x_arr, y_arr = p.compute_data(fig.x_range, global_samples=100)

out = Path("sine_samples.csv")
with out.open("w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["x", "y"])
    w.writerows(zip(x_arr.tolist(), y_arr.tolist()))

out, out.stat().st_size


## Applications (2–4 mini-projects)

Each mini-project is deterministic and intended to be understandable to advanced undergraduates / beginning grad students.


In [ ]:
x = sp.Symbol("x")
a = sp.Symbol("a")

sample_count = 2000
fig = SmartFigure(var=x, x_range=(-4, 4), samples=sample_count, show_now=False)
p = fig.plot(sp.exp(-a * x**2), name="gaussian")

def fwhm_from_samples(xv: np.ndarray, yv: np.ndarray) -> float:
    """Compute FWHM from samples of a unimodal symmetric peak at x=0."""
    if len(xv) == 0:
        return float("nan")
    ymax = float(np.max(yv))
    if ymax <= 0:
        return float("nan")
    half = 0.5 * ymax
    mid = int(np.argmin(np.abs(xv)))  # x≈0
    right = np.where(yv[mid:] <= half)[0]
    if len(right) == 0:
        return float("nan")
    j = mid + int(right[0])
    return float(2 * abs(xv[j]))

a_values = [0.25, 0.5, 1.0, 2.0, 4.0]
rows = []
for aval in a_values:
    fig.parameter_registry.get_param(a).value = aval
    xv, yv = p.compute_data(fig.x_range, global_samples=sample_count)
    rows.append((aval, fwhm_from_samples(xv, yv)))

rows


In [ ]:
# A quick visualization (Matplotlib backend)
mpl = MplBackend(figsize=(6, 3), dpi=110)
mpl.set_viewport(fig.x_range, (-0.1, 1.1))

# Plot three representative curves
for aval in (0.5, 1.0, 2.0):
    fig.parameter_registry.get_param(a).value = aval
    xv, yv = p.compute_data(fig.x_range, global_samples=sample_count)
    mpl.add_plot(f"a={aval}", xv, yv, Style(width=2.0))

mpl.request_redraw()

# Verify monotonic trend: FWHM decreases with a
fwhms = [w for _, w in rows]
assert all(fwhms[i] > fwhms[i+1] for i in range(len(fwhms)-1))


**What we learned:** increasing `a` makes the Gaussian narrower.

**How to adapt:** replace `exp(-a x^2)` by another parameterized family and compute a diagnostic (peak location, oscillation count, etc.).


### Application 1: Taylor approximation of `sin(x)`

**Motivating question:** How do Taylor polynomials approximate `sin(x)` on `[-π, π]`?

Workflow:
1) Generate Taylor polynomials of odd order.
2) Sample both functions.
3) Plot the absolute error.


In [76]:
f = sin(x)

def taylor_sin(order: int) -> sp.Expr:
    # SymPy series returns an O(x**n) term; remove it.
    return series(sin(x), x, 0, order+1).removeO()

orders = [1, 3, 5, 7, 9]
polys = {n: taylor_sin(n) for n in orders}

for n in orders:
    print("   Taylor polynomial of order n="+str(n)+":")
    display(polys[n])


   Taylor polynomial of order n=1:


x

   Taylor polynomial of order n=3:


-x**3/6 + x

   Taylor polynomial of order n=5:


x**5/120 - x**3/6 + x

   Taylor polynomial of order n=7:


-x**7/5040 + x**5/120 - x**3/6 + x

   Taylor polynomial of order n=9:


x**9/362880 - x**7/5040 + x**5/120 - x**3/6 + x

In [74]:
fig = SmartFigure(x_range=(-pi,pi), samples=1200)

In [75]:
fig.plot(f, name="sin(x)", style={"width": 2.0})

<Plot 'sin(x)': sin(x) | visible=True>

In [78]:
# Create polynomial plots (no parameters)
poly_plots = []
for n in orders:
    fig.plot(polys[n], name=f"taylor_{n}", style={"opacity": 0.9})


**What we learned:** on a fixed interval, higher-order Taylor polynomials reduce the error near 0, but can still grow toward the endpoints.

**How to adapt:** compare other approximations (Padé, Chebyshev) by plugging in different SymPy expressions.


### Application 2: A small “family of curves” mini-project — damped oscillations

**Motivating question:** How does a damping parameter change oscillations?

We consider
$$
f_{a}(x) = e^{-a x^2}\,\cos(6x).
$$

We’ll:
1) plot the curve,
2) compute a simple diagnostic (RMS amplitude on the interval),
3) show how the diagnostic changes with `a`.


In [ ]:
x = sp.Symbol("x")
a = sp.Symbol("a")

expr = sp.exp(-a * x**2) * sp.cos(6*x)

sample_count = 2000
fig = SmartFigure(var=x, x_range=(-4, 4), samples=sample_count, show_now=False)
p = fig.plot(expr, name="damped_cos")

def rms(y: np.ndarray) -> float:
    return float(np.sqrt(np.mean(np.square(y))))

a_values = np.linspace(0.0, 1.0, 6)
rms_values = []

for aval in a_values:
    fig.parameter_registry.get_param(a).value = float(aval)
    xv, yv = p.compute_data(fig.x_range, global_samples=sample_count)
    rms_values.append(rms(yv))

list(zip(a_values.tolist(), rms_values))


In [ ]:
# Quick plot of RMS vs parameter a
mpl = MplBackend(figsize=(5, 3), dpi=110)
mpl.set_viewport((float(a_values.min()), float(a_values.max())), (0, max(rms_values)*1.05))
mpl.add_plot("RMS amplitude", a_values, np.array(rms_values), Style(width=2.0))
mpl.request_redraw()

# Verify monotonic decrease (for this family / interval)
assert all(rms_values[i] >= rms_values[i+1] for i in range(len(rms_values)-1))


**What we learned:** damping reduces overall amplitude.

**How to adapt:** replace `cos(6x)` by another oscillatory term or change the damping envelope.


### Customization & configuration

At the `SmartFigure` level:
- `x_range`, `y_range`: viewport (floats)
- `samples`: global sampling count
- `parameter_registry`: share parameters across figures or isolate them

At the `Plot` level:
- `domain`: `(xmin, xmax)` or `VIEWPORT`
- `samples`: integer or `VIEWPORT`
- `style`: `Style(...)` or a mapping passed via `fig.plot(..., style=...)`

Tip: choose sample counts based on the highest frequency you expect to resolve; avoid excessive points by default.


In [ ]:
# Share one parameter registry across multiple figures
x = sp.Symbol("x")
a = sp.Symbol("a")

fig1 = SmartFigure(var=x, show_now=False)
shared = fig1.parameter_registry  # public attribute
fig2 = SmartFigure(var=x, parameter_registry=shared, show_now=False)

p1 = fig1.plot(sp.exp(-a*x**2), name="gauss1")
p2 = fig2.plot(sp.cos(a*x), name="cos2")

shared.get_param(a).value = 2.0

# Both figures read the same registry value when sampling
_, y1 = p1.compute_data(fig1.x_range, global_samples=200)
_, y2 = p2.compute_data(fig2.x_range, global_samples=200)

float(y1[int(len(y1)/2)]), float(y2[int(len(y2)/2)])


### Extending the package

#### 1) Implementing a custom backend (`PlotBackend`)

If you want to render somewhere else (another widget system, a file export, etc.), implement the `PlotBackend` protocol:
- `add_plot(name, x, y, style) -> handle`
- `update_plot(handle, x, y)`
- `apply_style(handle, style)`
- `remove_plot(handle)`
- `set_viewport(x_range, y_range)`
- `request_redraw()`

**Invariants your backend must preserve:**
- `x` and `y` are 1D NumPy arrays of the same length.
- Parameter symbols are scalars; if the underlying expression is constant in `x`, the model broadcasts it to the `x` grid.

Below is a tiny backend that records updates (useful for testing).


In [ ]:
# Example backend: record calls for tests (no UI)
from typing import List, Optional

class RecordingBackend:
    def __init__(self):
        self.calls: List[Tuple[str, Any]] = []

    def add_plot(self, name: str, x: np.ndarray, y: np.ndarray, style: Style) -> str:
        h = f"handle:{name}"
        self.calls.append(("add_plot", name, x.shape, y.shape, repr(style)))
        return h

    def update_plot(self, handle: str, x: np.ndarray, y: np.ndarray) -> None:
        self.calls.append(("update_plot", handle, x.shape, y.shape))

    def apply_style(self, handle: str, style: Style) -> None:
        self.calls.append(("apply_style", handle, repr(style)))

    def remove_plot(self, handle: str) -> None:
        self.calls.append(("remove_plot", handle))

    def set_viewport(self, x_range: Tuple[float, float], y_range: Optional[Tuple[float, float]]) -> None:
        self.calls.append(("set_viewport", x_range, y_range))

    def request_redraw(self) -> None:
        self.calls.append(("request_redraw",))

rb = RecordingBackend()
h = rb.add_plot("f", np.array([0.0, 1.0]), np.array([1.0, 2.0]), Style())
rb.update_plot(h, np.array([0.0]), np.array([0.0]))
rb.calls[:3]


#### 1b) `PlotlyTraceHandle` (for backend implementers)

`PlotlyTraceHandle` is a small dataclass used by `PlotlyBackend` to refer to traces stably (via Plotly's `uid`). Most users never need this directly, but it is useful when implementing tooling around the Plotly backend.


In [ ]:
from dataclasses import asdict

h = PlotlyTraceHandle(uid="example-uid-123")
asdict(h)


### Performance & scaling notes

Pragmatic guidance:
- **Sampling density** dominates cost: start with 200–1000 points, increase only when necessary.
- Compiling expressions can be expensive; reuse plots and update parameter values instead of recreating plots.
- For very expensive expressions, consider:
  - simplifying the SymPy expression first,
  - providing a custom numeric implementation (see `NamedFunction`),
  - reducing the viewport or sample count.

High-level profiling idea:
- Time `plot.compute_data(...)` in isolation for representative parameter values.


In [ ]:
# Micro-timing example (deterministic)
import time

x = sp.Symbol("x")
a = sp.Symbol("a")
expr = sp.exp(-a*x**2) * sp.cos(10*x)

sample_count = 2000
fig = SmartFigure(var=x, x_range=(-5, 5), samples=sample_count, show_now=False)
p = fig.plot(expr, name="f")

fig.parameter_registry.get_param(a).value = 1.0

t0 = time.perf_counter()
_ = p.compute_data(fig.x_range, global_samples=sample_count)
t1 = time.perf_counter()

print(f"compute_data: {(t1-t0)*1000:.1f} ms for {sample_count} samples")
